# Introduction
Hello!  In this tutorial we will scrape a more complicated webage from Wikipedia.  This is a continuation of [Part 1](https://onefortheroad.github.io/python/tutorial/2017/04/29/web-scraping-part-1/) where we learned the basics of web scraping.

When we left off Part 1, we had a *pandas* dataframe containing the Top 100 Canadian Beers. I'd like to add some **geospatial** information to our beer list so I can plan a pilgrimage to these fantastic breweries.  (Actually, we'll use this geospatial information in a future tutorial on visualization.)  Wikipedia's [List of Breweries in Canada](https://en.wikipedia.org/wiki/List_of_breweries_in_Canada) is a fine place to start.  Let's go!

## Contents


# 1. Import Libraries

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 2. Download the web page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_breweries_in_Canada'
page = requests.get(url)

# 3. Parse the html

In [4]:
soup = BeautifulSoup(page.content, 'lxml')